# N-GRAM 을 이용한 word embedding 구현
## YBIGTA _ 이상헌


## pytorch 튜토리얼 겸용
pytorch는 다음과 같은 프로세스로 진행된다 :
1. 데이터 정의
2. 모델 정의
3. Loss function 정의 
4. 역전파 계산
5. 계산을 통한 최적화.

이 프로세스를 ** N GRAM language modeling ** 으로 진행한다.


## 0.1. N-gram lang modeling?

## 0.2. What is word embedding?

word embedding이란 자신의 단어사전에 각 단어가 해당하는 *벡터*를 지칭하는 말이다. <br>

## 1. 데이터 정의

본 테스트는 필자가 가장 좋아하는 다이나믹 듀오의 *죽일놈*으로 테스트한다. <br>
보통 N-gram language model training 의 경우 100 million words가 넘어야 하므로 toy model이라고 받아들인다. <br>
본 테스트는 pytorch tutorial과 같이 trigram으로 진행한다. <br>
한국어의 경우는 영어와 달라 본 프로세스 전, 형태소 분석을 통해 조사를 제거해주는 작업이 필요하다. <br>
하지만 본 튜토리얼에서 해당 과정은 스킵한다.

In [24]:
test_sentence = """너 아까 나한테 왜 그랬어
너 또 왜 그러는데
내가 도대체 어디까지 맞춰야 돼
넌 맨날 그런 식이야 됐어 나 갈게

너는 뛰쳐나가 차 문을 부술 듯이 닫으면서
난 머리를 처박고 한숨 쉬어 핸들을 안으면서
이런 광경이 너무 익숙해 이젠
웬만한 싸움에는 상처도 잘 안 나 이젠
명품 쇼핑할 때처럼 너무 깐깐해 네 기준은
한번 화내면 뒤끝 장난 아냐 적어도 2주는 가니까
난 성격이 너무 물러서 
넌 항상 말해 남자니까 뒤로 좀 물러서
부담돼 네가 내게 결혼을 보채는 것도 
난 달인처럼 대화 화제를 돌리는 법도 많이 늘었어
넌 항상 추격하고 나는 도망쳐
솔직히 말할게 난 아직 준비 안 됐어
지쳤어 조금 널 향한 사랑은 도금이 
벗겨진 반지처럼 빛이 바랬어 
오늘은 이별을 말해야 될 것 같아 
지겹거든 너랑 다툴 때마다 항상 하는 말
내가 죽일 놈이지 뭐 우리가 어긋날 때면
전부 내 탓이지 뭐 마치 죄인인 것처럼
난 한걸음 물러서서 아무 말도 안 해
완벽한 너한테 난 항상 부족한 사람인 걸
처음에 쉬웠어 너와 함께라는 게 
난 너를 위해 내 자신을 숨기고 또 지웠어
사랑에 취해 네게 기대고
너란 퍼즐에 날 억지로 맞춰 끼웠어
하지만 이젠 나 지쳤어 
네가 만든 내게 난 숨이 막혀오는데
넌 점점 더 내게 바라는 게 많아졌어
마찰이 잦아졌어 네가 사준 
구두굽처럼 사랑이 닳아졌어
네 잔소리는 넥타이처럼 
내 목을 조여서 날 얌전하게 만들었지
그래서 그게 좋아 보였어 
그때 내 속은 한참 뒤틀리고 꼬였어
지금 난 널 만나기 전에 내가 너무 고파
이미 우리 사이 권태라는 벽은 너무 높아
내가 더 잘 할게 잘 할게 하며 발악했던 나지만
오늘은 말할래 이것이 너와 나의 마지막
내가 죽일 놈이지 뭐 우리가 어긋날 때면
전부 내 탓 이지 뭐 마치 죄인인 것처럼
난 한걸음 물러서서 아무 말도 안 해
완벽한 너한테 나 항상 부족한 사람인 걸
내가 잘 할게
내가 잘 할게란 말 이제 두 번 다시 안 할게
이 말 안 할래
너를 사랑해란 말 이제 두 번 다시 안 할게
내가 잘 할게
내게 잘해달란 말 이제 두 번 다시 안할게
이 말 안 할래
그동안 참아왔던 이별을 오늘은 네게 말할래

내가 죽일 놈이지 뭐 우리가 어긋날 때면
전부 내 탓 이지 뭐 마치 죄인인 것처럼
난 한걸음 물러서서 아무 말도 안 해
완벽한 너한테 나 항상 부족한 사람인 걸.""".split()

CONTEXT_SIZE = 2 # trigram이기 때문에 context size = 2이다.
# Trigram 변환
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

[(['너', '아까'], '나한테'), (['아까', '나한테'], '왜'), (['나한테', '왜'], '그랬어')]


In [25]:
# 각 단어를 index화하는 작업이 필요하다. 
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

word_to_ix

{'2주는': 102,
 '가니까': 39,
 '갈게': 151,
 '같아': 179,
 '걸': 100,
 '걸.': 113,
 '것': 161,
 '것도': 79,
 '것처럼': 112,
 '게': 149,
 '결혼을': 158,
 '고파': 72,
 '광경이': 173,
 '구두굽처럼': 122,
 '권태라는': 33,
 '그게': 98,
 '그동안': 136,
 '그때': 66,
 '그래서': 97,
 '그랬어': 77,
 '그러는데': 36,
 '그런': 137,
 '기대고': 92,
 '기준은': 109,
 '깐깐해': 140,
 '꼬였어': 41,
 '끼웠어': 90,
 '나': 164,
 '나는': 49,
 '나의': 14,
 '나지만': 29,
 '나한테': 194,
 '난': 155,
 '날': 94,
 '남자니까': 63,
 '내': 86,
 '내가': 85,
 '내게': 152,
 '너': 132,
 '너는': 82,
 '너란': 23,
 '너랑': 48,
 '너를': 103,
 '너무': 71,
 '너와': 24,
 '너한테': 80,
 '넌': 181,
 '널': 58,
 '네': 187,
 '네가': 130,
 '네게': 2,
 '넥타이처럼': 26,
 '놈이지': 7,
 '높아': 193,
 '늘었어': 117,
 '다시': 68,
 '다툴': 191,
 '닫으면서': 11,
 '달인처럼': 115,
 '닳아졌어': 52,
 '대화': 111,
 '더': 125,
 '도금이': 169,
 '도대체': 176,
 '도망쳐': 202,
 '돌리는': 127,
 '돼': 162,
 '됐어': 88,
 '될': 178,
 '두': 104,
 '뒤끝': 96,
 '뒤로': 195,
 '뒤틀리고': 143,
 '듯이': 89,
 '때마다': 180,
 '때면': 131,
 '때처럼': 108,
 '또': 73,
 '뛰쳐나가': 25,
 '마지막': 101,
 '마찰이': 154,
 '마치': 206,
 '막혀오는데': 44,
 '만나기': 5

In [26]:
"""
modeling을 위한 import
"""
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [27]:
"""
word embedding의 dimension을 말하는 것.
보통 정하는 기준이 무엇이지?
이 hyperparam을 정하는 기준은 다음에 다룬다.
ref ) https://arxiv.org/pdf/1601.00893.pdf
"""
EMBEDDING_DIM = 10


## 2. 모델 정의

모델을 정의한다. 모델을 정의하는 class는 nn.Module클래스를 input으로 받는데, <br>
그 nn.Module이란 layer와 해당 layer에서 forwarding하는 함수를 가진다. <br>
따라서 초기화 ` __init__ ` 에서 layer를 쌓고, <br>
forwarding하는 함수를 쌓는다. <br>
forwarding 하는 함수에는 초기화한 layer에 relu, batch등의 함수를 쌓아 만들고 <br>
loss function 까지 쌓는다.

In [28]:
"""
모델링.
모델에 대한 그림은 칠판에 그리고 설명한다.
1 layer 의 사이즈가 128인 이유는?
"""

class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs


## 3. loss function 정의

본 모델에서 loss function은 negative lognormal likelihood, 가장 단순한 모델을 사용한다. <br>


In [29]:
losses = [] # 각 epoch, 각 training step 마다 loss의 줄어듬을 보기위한 리스트
loss_function = nn.NLLLoss() # Negative log likelihood loss


## 4. 역전파 정의

역전파의 함수는 input으로 model을 받는다. <br>
따라서 모델을 먼저 초기화 하고, 그 다음 loss function안에 본 모델을 넣는다. <br>
learning rate와 함께 선언하고, 본 예에서는 SGD를 사용한다.

In [30]:
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE) 
optimizer = optim.SGD(model.parameters(), lr=0.001) # learning rate 는 보통 사용하는 0.001을 사용.

## 5. 최적화.

최적화는 다음과 같은 과정을 거친다. <br>
trigram model의 경우 index로 트레이닝을 한다. <br>
(본 index 트레이닝이 실질적으로 어떠한 효과가 있는지는 QnA에서 다룬다.  <br>
결국 input은 앞선 두 word의 index를 넣고, output은 training word의 index와 비교한다.

In [33]:
# 총 10번 트레이닝
for epoch in range(10):
#    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
    print(losses)  # The loss decreased every iteration over the training data!

[
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
, 
 55095.2969
[torch.FloatTensor of size 1]
]
[
 56721.0898
[torch.FloatTensor of size 1]
, 
 56721.